# Simple RAG for GitHub issues using Hugging Face Zephyr and LangChain

_Autora: [Maria Khalusova](https://github.com/MKhalusova)_
_Traductor: [Jan Leyva](https://github.com/JanLeyva)_

Aquesta llibreta ensenya com pots construir rapidament un RAG *(Retrieval Augmented Generation)* per a un projecte de *GitHub issues* utilitzant el model[`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), i LangChain.



**Que es un *RAG*?**

Un *RAG* es una técnica popular per corretgir el problema quan un *LLM* no te l'informació necessaries o bé perquè aquestes no estava en el seu conjunt de dades d'entrenament o bé per evitar al·lucinacions tot i haver-les vist abans. Aquestes dades poden ser de propietat privada, sensibles o com en l'exemple actualitzades sovint.

Si les teves dades son estatiques i no canvien sivint, hauries de considerar *fine-tune* un model de llenguatge. En moltes casos, encara que *fine-tune* pot ser costos i quan es fa repetidament pot comportar problemes (e.g. evitar que el model es desvi). Això passa quan el model te un comportament no desitjable.


**RAG (Retrieval Augmented Generation)** no requereix *fine-tine* (ajustar) el model. En comptes, el que fa el *RAG* es proporcionar mes context de dades rellevants al model i així pot generar millor respostes informades.


Aqui tenim una il·lustració d'un RAG:

![RAG diagram](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/rag-diagram.png)

* Les dades externes es converteixen en vectors *embedding* (representacions vecgtoritzada del text) amb un model que crea aquests *embeddings* diferent. Embeddings models son tipicament petits, així actualitzar els vectors creats es més ràpid, barat i fàcil que *fine-tune* el model.

* A la vegada, el fet de que el *fine-tune* no sigui necessari et dona mes llivertat a l'hora de canviar el *LLM* per un més potent quan estigui disponible. O canviar-lo per una versió mes petita i optima quan necessitis que la generació sigui mes ràpida.

Ilustrem com construir un *RAG* utilitzant un model lliure *LLM*, *embeddings* del model i LangChain.

Primer, instala les dependencies requerides:

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [2]:
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain

## Prepara les dades

En aquest example, carreguem totes les incidencies (ambdues obertes i tancades) desde [PEFT library's repo](https://github.com/huggingface/peft).

Primer, necessitem aconseguir [GitHub personal access token](https://github.com/settings/tokens?type=beta) to access the GitHub API.

In [ ]:
from getpass import getpass
ACCESS_TOKEN = getpass("YOUR_GITHUB_PERSONAL_TOKEN")

Després, carreguem totes les incidencies de [huggingface/peft](https://github.com/huggingface/peft) repositori:

- Per defecte, les *pull requests* son considerades incidencies també, per això escollim excloir-les de les dades editant la configuració `include_prs=False`
- Possa `state="all"` vol dir que carregarem ambdos tancats i oberts.

In [5]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(
    repo="huggingface/peft",
    access_token=ACCESS_TOKEN,
    include_prs=False,
    state="all"
)

docs = loader.load()

La llargada d'una incidencia de GitHub pot ser mes llarga de la capacitat màxima que pot admetre el *embedding*. Per això, si volem aplicar el *embbeding* a tot el contingut, hem de separar en troços del tamany adecuat les incidencies.

La manera més directe de fer això es separar el contingut en una mesura definida i marcar una sobreposició d'aquest. D'aquesta manera, mantenint una sobreposició entre la separació del text, mantenim el context semantic entre les diferents divisions del text. Per separar el text es recomana utilitzar [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), i per això es el que utilitzarem.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

## Crear els embeddings + retriever

Una vegada totes els documents tenen la llargada apropiada, podem crear una base de dades amb aquests *embbedings*.

Per crear els troços dels documents i fer els embeddings utilitzarem `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5). Hi ha moltes mes models de embbeding disponibles en la plataforma, pots fer-hi una ullada als que millor funcionen a [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

Per crear la base de dades vectoritzada, utilitzarem `FAISS`, una llibreria desenvolupada per Facebook AI. Aquesta llibreria ofereix similars resultats per busqueda i agrupament que una base de dades convencional. FAISS es actualmentr una de les llibreries mes utilitzades per busqueda de NN en base de dades gegants.

Utilitarem LangChain API per accedir ambdues llibreries FAISS i el model de *embedding*.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

Necessitem una manera de retornar els documents necessaris amb una demanda no estructurada (query unstructured). Per això, utilitzarem el métode `as_retriever` utilitzant la `db` com a suport:
- `search_type="similarity"` vol dir que volem fer una busqueda amb resultats similars entre la demanda/*query* i els documents.
- `search_kwargs={'k': 4}` ens retornara nomes els 4 resultats principals.

In [8]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

Tant la base de dades vectoritzada com el retornador de documents estan iniciats i configuratrs, el següent pas serà configurar la cadena del model.

## Carrega el model quantitzat

(un model quantitzat es un model que en comptes de expressar-lo en la seva màxima precisió els valors o fem una precisió menor (e.g. int8, in4))

Per aquest exemple, hem escollit [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), un model petit però potent.

Amb tant models que surten cada setmana, potser voldras canviar a un model més nou i més gran. La millor manera d'estar alerta dels ultims models de llicencia lliure es mirant [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

Per tal de fer més ràpida la generació de text carregarem el model quantitzat:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Configura la cadena del LLM

Finalment, tenim totes les peces que necessitavem per configurar la cedena del LLM.

Primer de tot, creem un generador de text `text_generation`utilitzant el model carregat i el seu *tokenitzador*.

Despres, crearem una plantilla de *prompt* - això ha de seguir el format del model, això vol dir que si en algun moment canvies els *checkpoint* del model, assegura't de canviar la plantilla per una apropiada també.

In [15]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Nota: _També pots utilitzar `tokenizer.apply_chat_template` per convertir la llista de missatges (com a diccionaris: `{'role': 'user', 'content': '(...)'}`) a str amb la plantilla apropiada._

Finalment, necessitem convinar `llm_chain` amb el que ens torna la base de dades vectoritzada per tal de crear la cadena del RAG. Per això, necessitem passar la pregunta original a traves del generador de text (LLM) juntament amb els documents mes rellevants que ens torni dels que hem fet el *embbeding* abans:

In [17]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


## Comparem els resultats

Mirem quina diferencia que hi ha generant les respostes amb el RAG a preguntes específiques.

In [18]:
question = "How do you combine multiple adapters?"

Primer de tot mirem quins resultats obtenim sense passar extra de context al model.

In [20]:
llm_chain.invoke({"context":"", "question": question})

" To combine multiple adapters, you need to ensure that they are compatible with each other and the devices you want to connect. Here's how you can do it:\n\n1. Identify the adapters you need: Determine which adapters you require to connect the devices you want to use together. For example, if you want to connect a USB-C device to an HDMI monitor, you may need a USB-C to HDMI adapter and a USB-C to USB-A adapter (if your computer only has USB-A ports).\n\n2. Connect the first adapter: Plug in the first adapter into the device you want to connect. For instance, if you're connecting a USB-C laptop to an HDMI monitor, plug the USB-C to HDMI adapter into the laptop's USB-C port.\n\n3. Connect the second adapter: Next, connect the second adapter to the first one. In this case, connect the USB-C to USB-A adapter to the USB-C port of the USB-C to HDMI adapter.\n\n4. Connect the final device: Finally, connect the device you want to use to the second adapter. For example, connect the HDMI cable

Com pots veuere el model interpreta la pregunta com si parlessim de adaptadors fisics d'ordinador, mentres que amb el contexte del PEFT, "adapters" es refereix als adaptadors de LoRa.

Anem a veure sia afeguint context de les incidencies de GitHub ajuda al model a donar una resposta més rellevant:

In [21]:
rag_chain.invoke(question)

" Based on the provided context, it seems that combining multiple adapters is still an open question in the community. Here are some possibilities:\n\n  1. Save the output from the base model and pass it to each adapter separately, as described in the first context snippet. This allows you to run multiple adapters simultaneously and reuse the output from the base model. However, this approach requires loading and running each adapter separately.\n\n  2. Export everything into a single PyTorch model, as suggested in the second context snippet. This would involve saving all the adapters and their weights into a single model, potentially making it larger and more complex. The advantage of this approach is that it would allow you to run all the adapters simultaneously without having to load and run them separately.\n\n  3. Merge multiple Lora adapters, as mentioned in the third context snippet. This involves adding multiple distinct, independent behaviors to a base model by merging multipl

Tal i com podem veure, afeguir context, realment ajuda al mateix model, i així retorna molt millor resposta i més relevant per el context que estavem preguntat.

Hem de tenir en compte, que combinar diferents "adapters" per generar text s'ha afeguit a la llibreria recentment, aquesta informació es pot trobar a la documentació, i per això podría ser beneficios afeguir la documentació a la base de dades vectoritzada havent fet els *embedding* corresponents. Per la pròxima itereció podria ser beneficios incluir això al RAG.